In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time
import re
import numpy as np
from PIL import Image
from io import BytesIO
import requests
import os
import json
from urllib.parse import unquote
import undetected_chromedriver as uc
# from selenium_stealth import stealth
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium import webdriver
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [19]:
data=pd.read_csv("Company Names with UIDs.csv")
company_list=data['COMPANY_NAME'].to_list()
company_list=company_list[1:3]
company_uids=data['UID'].to_list()
company_uids=company_uids[1:3]

In [23]:
json_list = []

for company,uid in zip(company_list,company_uids):
    retry_count = 0
    
    while retry_count < 3:
        print(uid)
        options = Options()
        options.profile = r"C:\Users\ajai\Desktop\Tor Browser\Browser\TorBrowser\Data\Browser\profile.default"
        options.binary_location = r"C:\Users\ajai\Desktop\Tor Browser\Browser\firefox.exe"
        driver = webdriver.Firefox(options=options)
        url = 'https://clutch.co/profile/' + str(company)
        print(url)
        driver.get(url)

        try:

            #read more
            try:
                button=driver.find_element(By.XPATH,'//*[@id="read_more"]')
                action = ActionChains(driver)
                action.move_to_element(button).click().perform()
                time.sleep(2)
            except Exception as e:
                print(e)

            # title
            try:
                title_content=driver.find_element(By.CLASS_NAME,'profile-header__title').get_attribute('data-subtitle')
                title=title_content.split('Client')[0].strip()
            except Exception:
                title=''
                print(Exception)

            #website-link
            try:
                website_url=driver.find_element(By.CLASS_NAME,'visit-website').get_attribute('href')
            except Exception:
                website_url=''
                print(Exception)

            #profile pic
            try:
                profile_src=driver.find_element(By.CSS_SELECTOR,'.profile-header__logotype img').get_attribute('src')
                response = requests.get(profile_src)
                profile_pic=f'gallery//images//{uid}//profile_pic//{company}_portfolio_pic.png'
                profile_pic1=profile_pic.replace("//",'/')
                directory = os.path.dirname(profile_pic)
                if not os.path.exists(directory):
                    os.makedirs(directory)
                with open(profile_pic, "wb") as f:
                    f.write(response.content)
            except NoSuchElementException:
                profile_pic1=''

            #Summary text
            try:
                summary_text=driver.find_element(By.CSS_SELECTOR,'.profile-summary__text').text.replace('\n','').replace('READ MORE...','')
            except Exception as e:
                print(e)
                summary_text=''

            #profile slogan
            try:
                profile_slogan=driver.find_element(By.CSS_SELECTOR,'h2.profile-summary__tagline').text
                print(profile_slogan)
            except Exception as e:
                print(e)
                print('Error Occured')
                profile_slogan=''

            #Star Rating
            try:
                average_review_rating=driver.find_element(By.CSS_SELECTOR,'span.sg-rating__number').text
            except NoSuchElementException:
                average_review_rating=''

            #Locations
            try:
                no_of_locations=driver.find_element(By.CSS_SELECTOR,'h3 > span').text
                no_of_locations=int(no_of_locations.replace("(",'').replace(')',''))
                for nol in list(range(0,no_of_locations)):
                    loc_button=driver.find_element(By.XPATH,f'//*[@id="location_{nol}"]/button')
                    driver.execute_script("arguments[0].scrollIntoView();", loc_button)
                    loc_button.click()
                    # action = ActionChains(driver)
                    # action.move_to_element(loc_button).click().perform()
                    time.sleep(2)

            except Exception as e:
                print(e)


            new_data={
                'uid':uid,
                'url':url,
                'website-url':website_url,
                'title':title,
                'profile-pic':profile_pic1,
                'profile-slogan':profile_slogan,
                'star-rating':average_review_rating,
                'profile-overview':summary_text
                # 'full-address':

            }
            json_list.append(new_data)

            break
                    
        except Exception as e:
            retry_count += 1
            print(f"Exception occurred: {e}. Retrying... Attempt {retry_count}")

            if retry_count >= 3:
                raise RuntimeError(f"Failed to process company {company} after 3 retries.") 

            # Wait for a few seconds before retrying
            time.sleep(2)
        finally:
            driver.quit()

json_data = json.dumps(json_list, indent=4)
print(json_data)


6660000002


https://clutch.co/profile/goji-labs
Launch Ventures, not Apps
Message: Element <button class="location-button profile-locations--item-button active" type="button"> is not clickable at point (283,9) because another element <div class="profile-header__wrap"> obscures it
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:187:5
ElementClickInterceptedError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:331:5
webdriverClickElement@chrome://remote/content/marionette/interaction.sys.mjs:162:11
interaction.clickElement@chrome://remote/content/marionette/interaction.sys.mjs:121:11
clickElement@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:214:29
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:97:31

6660000003
https://clutch.co/profile/mercury-development
Developing the Future
Message: Element <button class="location-but

In [12]:
summary_text.replace('\n','').replace('READ MORE...','')

"Naked. We don't just build apps, we turn ideas into app businesses.It takes a people-centric approach to building, branding & marketing your next big idea - so there's no cutting corners. That's why our team of creative problem-solving visionaires are in-house, and US-based. Whatever stage in your app journey, we come alongside you to become your strategic partner and acclerate your success."

In [17]:
int(no_of_locations.replace("(",'').replace(')',''))

1